Utilizando la data historica de la UFC, se hara un analisis descriptivo de la data, con un modelo de prediccion. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) 
import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
df = pd.read_csv("../input/ufcdata/data.csv")
preprocessed_data = pd.read_csv("../input/ufcdata/preprocessed_data.csv")
raw_fighter_details = pd.read_csv("../input/ufcdata/raw_fighter_details.csv")
raw_total_fight_data = pd.read_csv("../input/ufcdata/raw_total_fight_data.csv")

In [ ]:
df.describe()

Aqui podemos ver estadisticas descriptivas basicas del dataset

In [ ]:
df.isnull().sum() #Chequeando por missing values

In [ ]:
df2 = df[df.columns[df.columns.isin(['R_fighter','B_fighter','Referee','Winner','weight_class','no_of_rounds','title_bout','R_weight_lbs','R_Reach_cms','R_Height_cms','R_Stance','B_age','R_age'])]]
#Filtrando la data

La data original es bastante grande, por esto seria mejor limpiarla a conveniencia y utilizar las variables de mayor interés

In [ ]:
df2.head() #Viendo la nueva data

In [ ]:
df2.dropna(axis = 'rows', inplace = True ) # Dropping missing values, para un mejor analisis. Aun queda gran parte de la muestra por lo que se puede asumir que sigue siendo representativa.

Iniciemos el analisis descriptivo, algo muy importante para todo tipo de atletas es la edad. 

In [ ]:
df2['B_age'].describe() #Estadisticas descriptivas de la edad del luchador en la esquina azul


In [ ]:
df2['B_age'].describe() #Estadisticas descriptivas de la edad del luchador en la esquina roja

Con los datos anteriores podemos notar que la media de la edad de luchadores es de 29 años. El más joven tiene 18 años y el mas viejo en una pelea tuvo 51. 

In [ ]:
Azul = df.groupby(['B_age']).count()['Winner']
BAge = Azul.sort_values(axis=0, ascending = False)
BAge.head(3)
#Para ver la edad mas ganadora de la esquina azul

In [ ]:
Rojo = df.groupby(['R_age']).count()['Winner']
RAge = Rojo.sort_values(axis=0, ascending = False)
RAge.head(3)
#Edad mas ganadora de la esquina roja

Podemos ver que la edad mas ganadora difiere, en la esquina azul es 28 años, mientras que en la esquina roja es 30. 
Sin embargo, si vemos que la edad es una factor muy importante a la hora de la decidir quien ganara una pelea, podemos esperar que si un luchador de 28 se enfrenta con uno de 50, el de 28 tendrá probabilidades mas altas de ganar el encuentro. 

In [ ]:
sns.distplot(df2['B_age'])
sns.distplot(df2['R_age'])
#Distribucion de las edades

Para concluir, con las edades, vemos que la distribucion de ambas esquinas, asemeja una dist normal. 

Para seguir el analisis, veamos la correalacion que tienen las variables numericas de la data. 

In [ ]:
sns.heatmap(df2.corr()) #grafico de correlacion
df2.corr(method ='pearson') #tabla de correlacion

En esta tabla y grafica, podemos notar que si una pelea es por el titulo tiende a tener mayor numeros de rounds, algo que se puede esperar, dado que las peleas por titulo podriamos asumir que los luchadores estan mas preparados que en otras peleas. Aparte de esas variables no parece haber otra correlacion fuerte entre las variables mostradas. 

In [ ]:
((df[df["title_bout"]==1]["Referee"].value_counts())) #Cuenta los referees que fueron a las peleas por el titulo

En los referees que van a los titulos podemos ver una gran diferencia. Parece ser que solo utilizan a los mas preparados en las peleas mas importantes. Y al parecer John McCarthy, es el mejor referee de la UFC. 

In [ ]:
((df[df["title_bout"]==1]["Winner"].value_counts())) #numero de ganadores por color

In [ ]:
sns.countplot(x='Winner',data=df)
plt.title('Quien gana más?',fontsize=15) #grafica de los ganadores por color de esquina

En la tabla y grafica anterior, se puede ver que la esquina roja gana mucho más que la azul, algo sorpredente ya que el color no deberia tener mucho que ver con el outcome de la pelea. 

In [ ]:
((df[df["title_bout"]==1]["weight_class"].value_counts())) #Contar las peleas de titulo por peso

Asumiendo que mientras mas popular son las peleas de ese peso, más peleas por el titulo se deben hacer. 

In [ ]:
sns.countplot(x='title_bout',data=df)
plt.title('Clasificacion de peleas',fontsize=15)
#Que tantas peleas fueron por el titulo

Como era de esperarse la mayoria de las peleas que son por titulo son mucho menores que las normales. 

In [ ]:
sns.countplot(x = 'Winner', data = df2, hue = 'R_Stance')

Podemos ver que la mayoria de los luchadores son derechos (orthodox) en términos absolutos, mientras que una menor cantidad es zurdo (Southpaw), una cantidad casi insignificante son switch y open stance. 

Machine Learning

In [ ]:
x= df2[['R_age','B_age','title_bout','R_Reach_cms','R_Height_cms']]
y = df2['no_of_rounds']
#Creando las variables

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.4,
                                                   random_state = 101)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.4,
                                                   random_state = 101)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lm = LinearRegression()

In [ ]:
lm.fit(x_train, y_train)

In [ ]:
#Evaluacion del modelo 
print(lm.intercept_)

In [ ]:
coef = pd.DataFrame(lm.coef_, x.columns, columns=['Coeficientes'])
coef #Coeficientes del modelo 

Vemos que los mas influyente en el modelo es si la pelea es por el titulo, como habiamos visto antes en la correlacion de titulo y numero de rounds. 

        Predicciones
      

In [ ]:
predict = lm.predict(x_test)

In [ ]:
sns.distplot((y_test - predict),bins = 30)

In [ ]:
from sklearn import metrics 

In [ ]:
print ('MAE: ', metrics.mean_absolute_error(y_test,predict))
print ('MSE: ', metrics.mean_squared_error(y_test,predict))
print ('RMSE: ',np.sqrt(metrics.mean_squared_error(y_test,predict)))


La variacion de la desviacion estandar sin explicar es un 41% (RSME)